### Q1

In [1]:
!mlflow --version

mlflow, version 2.3.2


### Q2

In [2]:
%%bash

mkdir -p data
wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet -P data
wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet -P data
wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-03.parquet -P data

python preprocess_data.py --raw_data_path data --dest_path ./output
echo $(du -sh output/dv.pkl)

--2023-05-30 09:18:53--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.33.244.13, 13.33.244.164, 13.33.244.127, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.33.244.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1254291 (1.2M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_2022-01.parquet’

     0K .......... .......... .......... .......... ..........  4% 2.25M 1s
    50K .......... .......... .......... .......... ..........  8% 2.77M 0s
   100K .......... .......... .......... .......... .......... 12% 4.49M 0s
   150K .......... .......... .......... .......... .......... 16% 5.80M 0s
   200K .......... .......... .......... .......... .......... 20% 6.89M 0s
   250K .......... .......... .......... .......... .......... 24% 11.8M 0s
   300K .......... .......... .......... .......... .......... 28% 

152K output/dv.pkl


### Q3

In [3]:
!python train.py

2023/05/30 09:19:40 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/05/30 09:19:40 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [7]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
from train import EXPERIMENT_NAME as BASE

mlflow.set_tracking_uri("http://127.0.0.1:5000")
client = MlflowClient()
experiment = client.get_experiment_by_name(BASE)
run = client.search_runs(experiment_ids=experiment.experiment_id, 
                          run_view_type=ViewType.ACTIVE_ONLY, 
                          max_results=1)[0]
run.data.params['max_depth']

'10'

### Q4

In [8]:
!python hpo.py

2023/05/30 09:25:41 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
[I 2023-05-30 09:25:41,199] A new study created in memory with name: no-name-64fb0085-8ecf-4ad2-917b-cc808479781f
[I 2023-05-30 09:25:42,306] Trial 0 finished with value: 2.451379690825458 and parameters: {'n_estimators': 25, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 0 with value: 2.451379690825458.
[I 2023-05-30 09:25:42,494] Trial 1 finished with value: 2.4667366020368333 and parameters: {'n_estimators': 16, 'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 0 with value: 2.451379690825458.
[I 2023-05-30 09:25:43,491] Trial 2 finished with value: 2.449827329704216 and parameters: {'n_estimators': 34, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 2 with value: 2.449827329704216.
[I 2023-05-30 09:25:43,905] Trial 3 finished with value: 2.460983516558473 a

In [9]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
from hpo import EXPERIMENT_NAME as HPO

mlflow.set_tracking_uri("http://127.0.0.1:5000")
client = MlflowClient()
experiment = client.get_experiment_by_name(HPO)
run = client.search_runs(experiment_ids=experiment.experiment_id, 
                         run_view_type=ViewType.ACTIVE_ONLY, 
                         order_by=["metrics.val_rmse ASC"])[0]
run.data.metrics['val_rmse']

2.449827329704216

### Q5

In [10]:
!python register_model.py

2023/05/30 09:26:37 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
2023/05/30 09:26:40 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/alina/anaconda3/envs/mlops_zoomcamp/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
Successfully registered model 'random-forest-best'.
2023/05/30 09:26:50 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: random-forest-best, version 1
Created version '1' of model 'random-forest-best'.


In [11]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
from register_model import EXPERIMENT_NAME as BEST

mlflow.set_tracking_uri("http://127.0.0.1:5000")
client = MlflowClient()
experiment = client.get_experiment_by_name(BEST)
run = client.search_runs(experiment_ids=experiment.experiment_id, 
                         run_view_type=ViewType.ACTIVE_ONLY, 
                         order_by=["metrics.test_rmse ASC"])[0]
print(run.data.metrics['test_rmse'])

2.2854691906481364


In [12]:
import mlflow.pyfunc

from sklearn.metrics import mean_squared_error
from register_model import BEST_MODEL_NAME as BEST_MODEL
from register_model import load_pickle
from pathlib import Path

model_version = 1
data_path = "./output"
model = mlflow.pyfunc.load_model(model_uri=f"models:/{BEST_MODEL}/{model_version}")
X_test, y_test = load_pickle(Path(data_path) / "test.pkl")

test_rmse = mean_squared_error(y_test, model.predict(X_test), squared=False)
print(test_rmse)

2023/05/30 09:27:19 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


2.2854691906481364


In [13]:
client = MlflowClient()
client.transition_model_version_stage(
    name=BEST_MODEL, version=model_version, stage="Production"
)

<ModelVersion: aliases=[], creation_timestamp=1685428010411, current_stage='Production', description='', last_updated_timestamp=1685428055661, name='random-forest-best', run_id='7439b0267c1b468b9765038a3b6363fd', run_link='', source='mlflow-artifacts:/3/7439b0267c1b468b9765038a3b6363fd/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

### Q6

In [14]:
from pprint import pprint

client = MlflowClient()
for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)

{   'aliases': {},
    'creation_timestamp': 1685428010392,
    'description': '',
    'last_updated_timestamp': 1685428055661,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1685428010411, current_stage='Production', description='', last_updated_timestamp=1685428055661, name='random-forest-best', run_id='7439b0267c1b468b9765038a3b6363fd', run_link='', source='mlflow-artifacts:/3/7439b0267c1b468b9765038a3b6363fd/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>],
    'name': 'random-forest-best',
    'tags': {}}
